In [1]:
import sys
from pathlib import Path
sys.path.insert(1, str(Path().resolve().parent / 'Model'))
from model import Dataset, VAE
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

print('device = {}'.format(device))

params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 0}

# Define datasets and loaders
train_path = r'X:\DS Training Data\log stft samples\train.npy'
train_data = Dataset(train_path)
train_loader = torch.utils.data.DataLoader(train_data, drop_last=True,**params)

val_path = r'X:\DS Training Data\log stft samples\val.npy'
val_data = Dataset(val_path)
val_loader = torch.utils.data.DataLoader(val_data, drop_last=True, **params)

test_path = r'X:\DS Training Data\log stft samples\test.npy'
test_data = Dataset(test_path)
test_loader = torch.utils.data.DataLoader(test_data, drop_last=True, **params)

device = cuda:0


In [3]:
def loss_function(x_hat, x, mu, logvar, device, print=False):
    BCE = F.binary_cross_entropy_with_logits(x_hat.double(), 
                                             x.view(-1).double(), 
                                             reduction='mean',
                                             weight=torch.tensor([100]).to(device))
    KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))
    if print:
        print('BCE = {:2.2f}, KLD = {:2.2f}'.format(BCE, KLD))
    return BCE + KLD

### Define model

In [4]:
src_path = test_path = r'X:\DS Training Data\log stft samples\src.npy'

model = VAE(d=32, 
            src_path=src_path, 
            batch_size = params['batch_size'], 
            device=device).cuda()
print(model)

training with dropout=0.5
c:\Users\ewais\Documents\GitHub\DrumSynthesis\Drum Synthesis\Model\model.py:177: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.src = torch.tensor(self.src, device = device)
VAE(
  (enc): ResNetBigger(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequent

### Define criterion and optimizer

Issues:
- BCE loss is huge
    - Might need to normalize in -1, 1
- Don't know how tensors are being flattened
- Don't know if weights (in combine_samples() are being repeated along correct dimensions)
    - FIXED: looks like they are. Checked with torch.unique()
- x_hat values are consistently < -1000
    - FIXED: collapsed sample weights via mean instead of sum

In [5]:
criterion = loss_function

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3
)

max_epochs = 10

train_losses = []
val_losses = []

for epoch in range(max_epochs):
    model.train()
    train_loss = 0

    # Training
    for batch_idx, local_batch in enumerate(train_loader):
        #  Transfer to GPU
        local_batch = local_batch.to(device, dtype = torch.float32)

        # ================ forward ================
        x_hat, mu, logvar = model(local_batch)
        # print(x_hat[0:5])
        # print('mu: {}'.format(mu))
        # print('logvar {}'.format(logvar))
        # print('x_hat shape: {}'.format(x_hat.shape))
        # print('local_batch shape: {}'.format(local_batch.shape))
        loss = criterion(x_hat, local_batch, mu, logvar, device)
        # print('Loss computed successfully')
        train_loss += loss.item()
        # ================ backward ================
        loss.backward()
        optimizer.step()
        model.zero_grad()

        if batch_idx % 50 == 0:
            # print training update
            print('\nTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
                    epoch, batch_idx * len(local_batch), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

        # print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    
    # Potentially move this inside the for loop above
    train_losses.append(loss.item())

    # ================ validation ================
    model.eval()  # Put model in evaluation mode

    for batch_idx, local_batch in enumerate(val_loader):
        # Transfer to GPU
        local_batch = local_batch.to(device, dtype = torch.float32)

        # Model computations
        x_hat, mu, logvar = model(local_batch)
        loss = criterion(x_hat, local_batch, mu, logvar, device)

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Validation Loss: {}\n'.format(loss.item()))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n')
    val_losses.append(loss.item())


BCE = 204.29, KLD = 113.46

Train Epoch: 0 [0/2049 (0%)]	Loss: 317.745
Epoch 0: train loss: 317.74462181370046
BCE = 208.11, KLD = 112.06
Epoch 0: train loss: 320.1636098809828
BCE = 188.33, KLD = 110.65
Epoch 0: train loss: 298.98760554374087
BCE = 187.44, KLD = 109.64
Epoch 0: train loss: 297.07876407806
BCE = 194.25, KLD = 110.00
Epoch 0: train loss: 304.25258630706867
BCE = 182.77, KLD = 108.31
Epoch 0: train loss: 291.0806459455539
BCE = 192.04, KLD = 105.20
Epoch 0: train loss: 297.24292383484567
BCE = 186.38, KLD = 105.75
Epoch 0: train loss: 292.1334475783449
BCE = 183.91, KLD = 105.98
Epoch 0: train loss: 289.8901094997601
BCE = 174.79, KLD = 103.96
Epoch 0: train loss: 278.75506710162097
BCE = 184.68, KLD = 102.34
Epoch 0: train loss: 287.0207752130726
BCE = 192.32, KLD = 100.92
Epoch 0: train loss: 293.24101815119514
BCE = 185.15, KLD = 99.72
Epoch 0: train loss: 284.869656468337
BCE = 186.97, KLD = 99.91
Epoch 0: train loss: 286.88064741021975
BCE = 173.70, KLD = 95.89
Epoc

In [6]:
print(train_losses)

[108.70917902794838, 86.58048821457469, 77.04600905333038, 73.27018578272776, 71.7036877008201, 70.4390889681638, 70.85449719421595, 69.56563715504517, 69.60380504655413, 69.51264543665371]


In [7]:
import matplotlib.pyplot as plot

plt.plot(train_losses)

NameError: name 'plt' is not defined